In [1]:
import matplotlib.pyplot as plt
from moku.instruments import MultiInstrument
from moku.instruments import PIDController, WaveformGenerator
from moku.instruments import Datalogger
import os
import time

In [23]:
# WITH PID, RF @ 2dBm
# m = MultiInstrument('10.2.10.7', platform_id = 4, force_connect=True)
m = MultiInstrument('localhost:1025', platform_id = 4, force_connect=True)
try:
    datal = m.set_instrument(2, Datalogger)
    pid = m.set_instrument(3, PIDController)
    wg = m.set_instrument(4, WaveformGenerator)
    
    # set up connections for multiinstrument mode
    connections = [
        dict(source="Input3", destination="Slot3InA"),  # input 3 -> PID input
        dict(source="Slot3OutA", destination="Slot4InA"),  # PID output -> waveform generator input (for amplitude modulation)
        dict(source="Slot4OutA", destination="Output4"),  # waveform generator (RF) -> Output 4
        dict(source="Input1", destination="Slot2InA"), # input 1 -> data logger input
    ]
    print(m.set_connections(connections=connections))

    m.set_frontend(channel = 1, impedance='1MOhm', coupling='DC', attenuation ='-20dB') # configure input port 1 - sensor and power meter
    m.set_frontend(channel = 3, impedance='1MOhm', coupling='DC', attenuation='0dB') # configure input port 3 PID input

    # additional code if you don't want to start right away and want to wait for sometime----------------
    # Turn OFF the waveform generator initially
    wg.generate_waveform(channel=1, type='DC', dc_level=0, amplitude=0)

    # print("Waiting 1 hour before starting data collection...")
    # time.sleep(30)  # **Wait 1 hour**
    # ------------------------------------------------------------------------------------------------

    # waveform generator and amplitude modulation--------------------------------------------------------------
    wg.generate_waveform(channel=1, type='Sine', frequency=80e6, amplitude=0.79621/2)  # V_peak = Vpp/2
    # some commonly used values: 0 dBm : 0.63246 Vpp
    #                            1 dBm : 0.70963 Vpp
    #                            2 dBm : 0.79621 Vpp
    #                            6 dBm : 1.2619 Vpp
    #                            10 dBm: 2.0000 Vpp
    wg.set_modulation(channel=1, type='Amplitude', source='InputA', depth=100) # amplitude modulation controlled by PID
    # -------------------------------------------------------------------------------------------------------------

    # PID------------------------------------------------------------------------------------------------
    # pid.set_by_frequency(channel = 1, prop_gain = 4, int_crossover = 10e3)
    # pid.set_by_frequency(channel = 1, prop_gain = 20, int_crossover = 1e2)
    pid.set_by_frequency(channel = 1, prop_gain = 23, int_crossover = 60)
    #prop_gain = 38, int_crossover = 36e3, diff_crossover = 240e3)  # P gain in dB, I and D in crossover freq in Hz
    pid.set_control_matrix(channel = 1, input_gain1 = 1, input_gain2 = 0)  # input matrix
    pid.set_input_offset(channel = 1, offset = -0.015) # bias
    pid.enable_output(channel = 1, signal = True, output = True)
    #----------------------------------------------------------------------------------------------------
    time.sleep(5)  # wait 15 seconds while the signal stabilizes
    print("Just finished waiting 30 seconds, now starting data collection...")
    
    # Data collection settings
    num_iterations = 1          # number of times when you want to collect data
    logging_duration = 10       # Each logging session lasts 60 seconds
    interval_between_logs = 600 - logging_duration  # Wait 110 seconds before next collection
    
    for i in range(num_iterations):
        print(f"Starting data collection {i+1}/{num_iterations}...")

        # Generate a unique filename using timestamp
        filename = f"PI.2dBm_{time.strftime('%Y%m%d_%H%M%S')}.li"

        # Start logging
        datal.enable_input(channel=1, enable=True)
        datal.set_samplerate(20)
        datal.set_acquisition_mode(mode='Precision')
        logFile = datal.start_logging(duration=logging_duration, file_name_prefix=filename)

        # Track progress
        complete = False
        while not complete:
            time.sleep(0.5)  # Sleep to avoid excessive polling
            progress = datal.logging_progress()
            complete = progress['complete']
            # if 'time_remaining' in progress:
            #     print(f"Remaining time {progress['time_remaining']} seconds")

        # Download log file
        # local_path = os.path.join(os.getcwd(), filename)
        # datal.download("persist", logFile['file_name'], local_path)
        # print(f"Downloaded log file to: {local_path}")

        # Wait before the next logging session
        if i < num_iterations - 1:
            print(f"Waiting {interval_between_logs} seconds before next collection...\n")
            time.sleep(interval_between_logs)

    print("Data collection complete!")

except Exception as e:
    print(f'Exception occurred: {e}')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    m.set_connections(connections=connections) # this comment is what the liquid instruments tech guy added
    m.relinquish_ownership()

i = WaveformGenerator('localhost:1025', force_connect=True)
i.generate_waveform(channel=4, type='DC', dc_level=0, amplitude=0)
i.relinquish_ownership()

[{'destination': 'Slot2InA', 'source': 'Input1'}, {'destination': 'Slot3InA', 'source': 'Input3'}, {'destination': 'Slot3InB', 'source': 'Slot2OutB'}, {'destination': 'Slot4InA', 'source': 'Slot3OutA'}, {'destination': 'Slot4InB', 'source': 'Slot3OutB'}, {'destination': 'Output4', 'source': 'Slot4OutA'}]
Just finished waiting 30 seconds, now starting data collection...
Starting data collection 1/1...
Data collection complete!


In [27]:
# without PID, RF @ 2dBm
# m = MultiInstrument('10.2.10.7', platform_id = 4, force_connect=True)
m = MultiInstrument('localhost:1025', platform_id = 4, force_connect=True)
try:
    datal = m.set_instrument(2, Datalogger)
    wg = m.set_instrument(4, WaveformGenerator)
    
    # set up connections for multiinstrument mode
    connections = [
        dict(source="Slot4OutA", destination="Output4"),  # waveform generator (RF) -> Output 4
        dict(source="Input1", destination="Slot2InA"), # input 1 -> data logger input
    ]
    print(m.set_connections(connections=connections))

    m.set_frontend(channel = 1, impedance='1MOhm', coupling='DC', attenuation ='-20dB') # configure input port 1 - sensor and power meter

    # additional code if you don't want to start right away and want to wait for sometime----------------
    # Turn OFF the waveform generator initially
    wg.generate_waveform(channel=1, type='DC', dc_level=0, amplitude=0)

    # print("Waiting 1 hour before starting data collection...")
    # time.sleep(30)  # **Wait 1 hour**
    # ------------------------------------------------------------------------------------------------

    # waveform generator and amplitude modulation--------------------------------------------------------------
    wg.generate_waveform(channel=1, type='Sine', frequency=80e6, amplitude=0.79621/2)  # V_peak = Vpp/2
    # some commonly used values: 0 dBm : 0.63246 Vpp
    #                            1 dBm : 0.70963 Vpp
    #                            2 dBm : 0.79621 Vpp
    #                            6 dBm : 1.2619 Vpp
    #                            10 dBm: 2.0000 Vpp
    # -------------------------------------------------------------------------------------------------------------

    time.sleep(5)  # wait 15 seconds while the signal stabilizes
    print("Just finished waiting 5 seconds, now starting data collection...")
    
    # Data collection settings
    num_iterations = 18          # number of times when you want to collect data
    logging_duration = 60       # Each logging session lasts 60 seconds
    interval_between_logs = 600 - logging_duration  # Wait 110 seconds before next collection
    
    for i in range(num_iterations):
        print(f"Starting data collection {i+1}/{num_iterations}...")

        # Generate a unique filename using timestamp
        filename = f"noPI.2dBm_{time.strftime('%Y%m%d_%H%M%S')}.li"

        # Start logging
        datal.enable_input(channel=1, enable=True)
        datal.set_samplerate(10)
        datal.set_acquisition_mode(mode='Precision')
        logFile = datal.start_logging(duration=logging_duration, file_name_prefix=filename)

        # Track progress
        complete = False
        while not complete:
            time.sleep(0.5)  # Sleep to avoid excessive polling
            progress = datal.logging_progress()
            complete = progress['complete']
            # if 'time_remaining' in progress:
            #     print(f"Remaining time {progress['time_remaining']} seconds")

        # Download log file
        # local_path = os.path.join(os.getcwd(), filename)
        # datal.download("persist", logFile['file_name'], local_path)
        # print(f"Downloaded log file to: {local_path}")

        # Wait before the next logging session
        if i < num_iterations - 1:
            print(f"Waiting {interval_between_logs} seconds before next collection...\n")
            time.sleep(interval_between_logs)

    print("Data collection complete!")

except Exception as e:
    print(f'Exception occurred: {e}')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    m.set_connections(connections=connections) # this comment is what the liquid instruments tech guy added
    m.relinquish_ownership()

i = WaveformGenerator('localhost:1025', force_connect=True)
i.generate_waveform(channel=4, type='DC', dc_level=0, amplitude=0)
i.relinquish_ownership()

[{'destination': 'Slot2InA', 'source': 'Input1'}, {'destination': 'Output4', 'source': 'Slot4OutA'}]
Just finished waiting 5 seconds, now starting data collection...
Starting data collection 1/18...
Waiting 540 seconds before next collection...

Starting data collection 2/18...
Waiting 540 seconds before next collection...

Starting data collection 3/18...
Waiting 540 seconds before next collection...

Starting data collection 4/18...
Waiting 540 seconds before next collection...

Starting data collection 5/18...
Waiting 540 seconds before next collection...

Starting data collection 6/18...
Waiting 540 seconds before next collection...

Starting data collection 7/18...
Waiting 540 seconds before next collection...

Starting data collection 8/18...
Waiting 540 seconds before next collection...

Starting data collection 9/18...
Waiting 540 seconds before next collection...

Starting data collection 10/18...
Waiting 540 seconds before next collection...

Starting data collection 11/18...

In [10]:
# without PID/SINARA, no RF
# m = MultiInstrument('10.2.10.7', platform_id = 4, force_connect=True)
i = WaveformGenerator('localhost:1025', force_connect=True)
i.generate_waveform(channel=4, type='DC', dc_level=0, amplitude=0)
i.relinquish_ownership()

m = MultiInstrument('localhost:1025', platform_id = 4, force_connect=True)
try:
    datal = m.set_instrument(2, Datalogger)
    
    # set up connections for multiinstrument mode
    connections = [
        dict(source="Input1", destination="Slot2InA"), # input 1 -> data logger input
    ]
    print(m.set_connections(connections=connections))

    m.set_frontend(channel = 1, impedance='1MOhm', coupling='DC', attenuation ='-40dB') # configure input port 1 - sensor and power meter

    # additional code if you don't want to start right away and want to wait for sometime----------------
    # print("Waiting 1 hour before starting data collection...")
    # time.sleep(30)  # **Wait 1 hour**
    # ------------------------------------------------------------------------------------------------

    time.sleep(5)  # wait 15 seconds while the signal stabilizes
    print("Just finished waiting 5 seconds, now starting data collection...")
    
    # Data collection settings
    num_iterations = 18          # number of times when you want to collect data
    logging_duration = 60       # Each logging session lasts 60 seconds
    interval_between_logs = 600 - logging_duration  # Wait 110 seconds before next collection
    
    for i in range(num_iterations):
        print(f"Starting data collection {i+1}/{num_iterations}...")

        # Generate a unique filename using timestamp
        filename = f"noSINARA_{time.strftime('%Y%m%d_%H%M%S')}.li"

        # Start logging
        datal.enable_input(channel=1, enable=True)
        datal.set_samplerate(20)
        datal.set_acquisition_mode(mode='Precision')
        logFile = datal.start_logging(duration=logging_duration, file_name_prefix=filename)

        # Track progress
        complete = False
        while not complete:
            time.sleep(0.5)  # Sleep to avoid excessive polling
            progress = datal.logging_progress()
            complete = progress['complete']
            # if 'time_remaining' in progress:
            #     print(f"Remaining time {progress['time_remaining']} seconds")

        # Download log file
        # local_path = os.path.join(os.getcwd(), filename)
        # datal.download("persist", logFile['file_name'], local_path)
        # print(f"Downloaded log file to: {local_path}")

        # Wait before the next logging session
        if i < num_iterations - 1:
            print(f"Waiting {interval_between_logs} seconds before next collection...\n")
            time.sleep(interval_between_logs)

    print("Data collection complete!")

except Exception as e:
    print(f'Exception occurred: {e}')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    m.set_connections(connections=connections) # this comment is what the liquid instruments tech guy added
    m.relinquish_ownership()

[{'destination': 'Slot2InA', 'source': 'Input1'}]
Just finished waiting 5 seconds, now starting data collection...
Starting data collection 1/1...
Data collection complete!


In [13]:
# WITH SINARA, RF @ 2dBm
# m = MultiInstrument('10.2.10.7', platform_id = 4, force_connect=True)
m = MultiInstrument('localhost:1025', platform_id = 4, force_connect=True)
try:
    datal = m.set_instrument(2, Datalogger)
    wg = m.set_instrument(4, WaveformGenerator)
    
    # set up connections for multiinstrument mode
    connections = [
        dict(source="Slot4OutA", destination="Output4"),  # waveform generator (RF) -> Output 4
        dict(source="Input1", destination="Slot2InA"), # input 1 -> data logger input
    ]
    print(m.set_connections(connections=connections))

    m.set_frontend(channel = 1, impedance='1MOhm', coupling='DC', attenuation ='-40dB') # configure input port 1 - sensor and power meter

    # additional code if you don't want to start right away and want to wait for sometime----------------
    # Turn OFF the waveform generator initially
    #wg.generate_waveform(channel=1, type='DC', dc_level=0, amplitude=0)

    # print("Waiting 1 hour before starting data collection...")
    # time.sleep(30)  # **Wait 1 hour**
    # ------------------------------------------------------------------------------------------------

    # waveform generator and amplitude modulation--------------------------------------------------------------
    wg.generate_waveform(channel=1, type='Sine', frequency=80e6, amplitude=0.79621) # CAUTION: amplitude here is peak-to-peak and NOT mean value to peak
    # some commonly used values: 0 dBm : 0.63246 Vpp
    #                            1 dBm : 0.70963 Vpp
    #                            2 dBm : 0.79621 Vpp
    #                            6 dBm : 1.2619 Vpp
    #                            10 dBm: 2.0000 Vpp
    # -------------------------------------------------------------------------------------------------------------

    time.sleep(30)  # wait 30 seconds while the signal stabilizes
    print("Just finished waiting 30 seconds, now starting data collection...")
    
    # Data collection settings
    num_iterations = 1          # number of times when you want to collect data
    logging_duration = 120       # Each logging session lasts 60 seconds
    interval_between_logs = 600 - logging_duration  # Wait 110 seconds before next collection
    
    for i in range(num_iterations):
        print(f"Starting data collection {i+1}/{num_iterations}...")

        # Generate a unique filename using timestamp
        filename = f"SINARA.2dBm_{time.strftime('%Y%m%d_%H%M%S')}.li"

        # Start logging
        datal.enable_input(channel=1, enable=True)
        datal.set_samplerate(20)
        datal.set_acquisition_mode(mode='Precision')
        logFile = datal.start_logging(duration=logging_duration, file_name_prefix=filename)

        # Track progress
        complete = False
        while not complete:
            time.sleep(0.5)  # Sleep to avoid excessive polling
            progress = datal.logging_progress()
            complete = progress['complete']
            # if 'time_remaining' in progress:
            #     print(f"Remaining time {progress['time_remaining']} seconds")

        # Download log file
        # local_path = os.path.join(os.getcwd(), filename)
        # datal.download("persist", logFile['file_name'], local_path)
        # print(f"Downloaded log file to: {local_path}")

        # Wait before the next logging session
        if i < num_iterations - 1:
            print(f"Waiting {interval_between_logs} seconds before next collection...\n")
            time.sleep(interval_between_logs)

    print("Data collection complete!")

except Exception as e:
    print(f'Exception occurred: {e}')
finally:
    # Close the connection to the Moku device
    # This ensures network resources and released correctly
    m.set_connections(connections=connections) # this comment is what the liquid instruments tech guy added
    m.relinquish_ownership()

i = WaveformGenerator('localhost:1025', force_connect=True)
i.generate_waveform(channel=4, type='DC', dc_level=0, amplitude=0)
i.relinquish_ownership()

[{'destination': 'Slot2InA', 'source': 'Input1'}, {'destination': 'Output4', 'source': 'Slot4OutA'}]
Just finished waiting 30 seconds, now starting data collection...
Starting data collection 1/1...
Data collection complete!
